In [1]:
import pandas as pd
import json
chunksize = 10 ** 6

In [59]:
coordinates_raw=pd.read_csv('coordinates.csv', encoding = "ISO-8859-1",chunksize=chunksize, usecols = ["id", "latitude", "longitude", "speed", "trip_id", "timestamp_txt"])
trips_raw=pd.read_csv('trips.csv', encoding = "ISO-8859-1",chunksize=chunksize, usecols = ["id", "avg_speed", "duration", "mode", "mode_description", "timestamp_start", "timestamp_end"])

In [60]:
processed_chunked_coordinates = []
processed_chunked_trips = []

In [61]:
def cord_chunk_preprocess(cord_chunk):
    cord_chunk.dropna(how='any')
    cord_chunk['coordinate_ftimestamp'] = pd.to_datetime(cord_chunk['timestamp_txt']).values.astype(float)
    return cord_chunk

In [62]:
def trip_chunk_preprocess(trip_chunk):
    trip_chunk.dropna(how='any')
    valid_modes = [0, 1, 3]
    trip_chunk = trip_chunk.loc[trip_chunk['mode'].isin(valid_modes)]   
    trip_chunk['mode'] = trip_chunk['mode'].replace(3, 2)
    trip_chunk['trip_ftimestamp_start'] = pd.to_datetime(trip_chunk['timestamp_start']).values.astype(float)
    trip_chunk['trip_ftimestamp_end'] = pd.to_datetime(trip_chunk['timestamp_end']).values.astype(float)
    return trip_chunk

In [63]:
for cord_chunk in coordinates_raw:
    chunk_filter = cord_chunk_preprocess(cord_chunk)
    processed_chunked_coordinates.append(cord_chunk)
processed_coord_df = pd.concat(processed_chunked_coordinates)

In [64]:
for trip_chunk in trips_raw:
    chunk_filter = trip_chunk_preprocess(trip_chunk)
    processed_chunked_trips.append(chunk_filter)
processed_trips_df = pd.concat(processed_chunked_trips)

F:\Concor\IDE\Anaconda\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
F:\Concor\IDE\Anaconda\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
F:\Concor\IDE\Anaconda\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexi

In [65]:
#Preparing same trips on both the trips and coord data frame.
unq_trips = []
unq_trips = processed_trips_df['id']
processed_coord_df = processed_coord_df.loc[processed_coord_df['trip_id'].isin(unq_trips)]
trip_cord_count = processed_coord_df.groupby('trip_id')['id'].nunique()
unq_trips = []
unq_trips = processed_trips_df['id']
processed_trips_df = processed_trips_df.loc[processed_trips_df['id'].isin(trip_cord_count.index.values)]

In [66]:
processed_coord_df.head()

,id,latitude,longitude,speed,trip_id,timestamp_txt,coordinate_ftimestamp
0,72368,45.528405,-73.640266,9.9,3444630,2016-10-15T16:06:23-04,1.476562e+18
1,188446,45.538669,-73.559904,14.6,3384546,2016-10-17T12:51:50-04,1.476723e+18
2,685872,45.534879,-73.569446,4.0,3151626,2016-10-18T08:59:06-04,1.476796e+18
13,8567215,45.515033,-73.575061,7.0,3234465,2016-10-24T17:24:59-04,1.477344e+18
16,10819477,45.507221,-73.575728,4.1,3055391,2016-10-26T08:50:25-04,1.477486e+18


In [68]:
processed_trips_df.head()

,id,avg_speed,duration,mode,mode_description,timestamp_start,timestamp_end,trip_ftimestamp_start,trip_ftimestamp_end
11,1693055,20.8,1719,2.0,Automobile,2016-10-25T18:37:56.000,2016-10-25T19:06:35.000,1.477421e+18,1.477422e+18
14,1629365,78.1,2266,2.0,Automobile,2016-10-23T11:24:33.000,2016-10-23T12:02:19.000,1.477222e+18,1.477224e+18
16,1966430,12.4,831,2.0,Automobile,2016-10-27T18:39:24.000,2016-10-27T18:53:15.000,1.477594e+18,1.477594e+18
21,1467160,34.8,2336,2.0,Automobile,2016-10-25T12:06:47.000,2016-10-25T12:45:43.000,1.477397e+18,1.477400e+18
23,3488393,3.7,551,0.0,À pied,2016-10-23T16:26:08.000,2016-10-23T16:35:19.000,1.477240e+18,1.477241e+18


In [69]:
Merged_Trip_Cord_Df = pd.merge(left=processed_trips_df,right=processed_coord_df, left_on='id', right_on='trip_id')

In [70]:
Merged_Trip_Cord_Df.head()

,id_x,avg_speed,duration,mode,mode_description,timestamp_start,timestamp_end,trip_ftimestamp_start,trip_ftimestamp_end,id_y,latitude,longitude,speed,trip_id,timestamp_txt,coordinate_ftimestamp
0,1693055,20.8,1719,2.0,Automobile,2016-10-25T18:37:56.000,2016-10-25T19:06:35.000,1.477421e+18,1.477422e+18,10262787,45.513605,-73.714926,9.0,1693055,2016-10-25T19:06:15-04,1.477437e+18
1,1693055,20.8,1719,2.0,Automobile,2016-10-25T18:37:56.000,2016-10-25T19:06:35.000,1.477421e+18,1.477422e+18,10262788,45.512483,-73.711553,12.9,1693055,2016-10-25T19:05:35-04,1.477437e+18
2,1693055,20.8,1719,2.0,Automobile,2016-10-25T18:37:56.000,2016-10-25T19:06:35.000,1.477421e+18,1.477422e+18,10262789,45.512133,-73.710771,6.0,1693055,2016-10-25T19:05:28-04,1.477437e+18
3,1693055,20.8,1719,2.0,Automobile,2016-10-25T18:37:56.000,2016-10-25T19:06:35.000,1.477421e+18,1.477422e+18,10262790,45.512390,-73.710525,6.5,1693055,2016-10-25T19:05:26-04,1.477437e+18
4,1693055,20.8,1719,2.0,Automobile,2016-10-25T18:37:56.000,2016-10-25T19:06:35.000,1.477421e+18,1.477422e+18,10262791,45.512215,-73.710059,5.4,1693055,2016-10-25T19:05:18-04,1.477437e+18


In [71]:
print (processed_trips_df.columns)
print (processed_coord_df.columns)

Index(['id', 'avg_speed', 'duration', 'mode', 'mode_description',
       'timestamp_start', 'timestamp_end', 'trip_ftimestamp_start',
       'trip_ftimestamp_end'],
      dtype='object')
Index(['id', 'latitude', 'longitude', 'speed', 'trip_id', 'timestamp_txt',
       'coordinate_ftimestamp'],
      dtype='object')


In [73]:
merged_cols = Merged_Trip_Cord_Df.columns.tolist()
print(merged_cols)
filtered_cols = ['trip_id', 'avg_speed', 'duration', 'mode', 'mode_description', 'timestamp_start', 'timestamp_end', 'trip_ftimestamp_start', 'trip_ftimestamp_end', 'id_y', 'latitude', 'longitude', 'speed', 'timestamp_txt', 'coordinate_ftimestamp']

['id_x', 'avg_speed', 'duration', 'mode', 'mode_description', 'timestamp_start', 'timestamp_end', 'trip_ftimestamp_start', 'trip_ftimestamp_end', 'id_y', 'latitude', 'longitude', 'speed', 'trip_id', 'timestamp_txt', 'coordinate_ftimestamp']


In [74]:
Merged_Trip_Cord_Df = Merged_Trip_Cord_Df[filtered_cols]

In [75]:
Merged_Trip_Cord_Df.head()

,trip_id,avg_speed,duration,mode,mode_description,timestamp_start,timestamp_end,trip_ftimestamp_start,trip_ftimestamp_end,id_y,latitude,longitude,speed,timestamp_txt,coordinate_ftimestamp
0,1693055,20.8,1719,2.0,Automobile,2016-10-25T18:37:56.000,2016-10-25T19:06:35.000,1.477421e+18,1.477422e+18,10262787,45.513605,-73.714926,9.0,2016-10-25T19:06:15-04,1.477437e+18
1,1693055,20.8,1719,2.0,Automobile,2016-10-25T18:37:56.000,2016-10-25T19:06:35.000,1.477421e+18,1.477422e+18,10262788,45.512483,-73.711553,12.9,2016-10-25T19:05:35-04,1.477437e+18
2,1693055,20.8,1719,2.0,Automobile,2016-10-25T18:37:56.000,2016-10-25T19:06:35.000,1.477421e+18,1.477422e+18,10262789,45.512133,-73.710771,6.0,2016-10-25T19:05:28-04,1.477437e+18
3,1693055,20.8,1719,2.0,Automobile,2016-10-25T18:37:56.000,2016-10-25T19:06:35.000,1.477421e+18,1.477422e+18,10262790,45.512390,-73.710525,6.5,2016-10-25T19:05:26-04,1.477437e+18
4,1693055,20.8,1719,2.0,Automobile,2016-10-25T18:37:56.000,2016-10-25T19:06:35.000,1.477421e+18,1.477422e+18,10262791,45.512215,-73.710059,5.4,2016-10-25T19:05:18-04,1.477437e+18


In [76]:
Merged_Trip_Cord_Df.rename(columns={'timestamp_start': 'trip_timestamp_start', 'timestamp_end': 'trip_timestamp_end', 'id_y': 'coordinate_id', 'timestamp_txt': 'coordinate_timestamp'}, inplace=True)

In [77]:
Merged_Trip_Cord_Df.head()

,trip_id,avg_speed,duration,mode,mode_description,trip_timestamp_start,trip_timestamp_end,trip_ftimestamp_start,trip_ftimestamp_end,coordinate_id,latitude,longitude,speed,coordinate_timestamp,coordinate_ftimestamp
0,1693055,20.8,1719,2.0,Automobile,2016-10-25T18:37:56.000,2016-10-25T19:06:35.000,1.477421e+18,1.477422e+18,10262787,45.513605,-73.714926,9.0,2016-10-25T19:06:15-04,1.477437e+18
1,1693055,20.8,1719,2.0,Automobile,2016-10-25T18:37:56.000,2016-10-25T19:06:35.000,1.477421e+18,1.477422e+18,10262788,45.512483,-73.711553,12.9,2016-10-25T19:05:35-04,1.477437e+18
2,1693055,20.8,1719,2.0,Automobile,2016-10-25T18:37:56.000,2016-10-25T19:06:35.000,1.477421e+18,1.477422e+18,10262789,45.512133,-73.710771,6.0,2016-10-25T19:05:28-04,1.477437e+18
3,1693055,20.8,1719,2.0,Automobile,2016-10-25T18:37:56.000,2016-10-25T19:06:35.000,1.477421e+18,1.477422e+18,10262790,45.512390,-73.710525,6.5,2016-10-25T19:05:26-04,1.477437e+18
4,1693055,20.8,1719,2.0,Automobile,2016-10-25T18:37:56.000,2016-10-25T19:06:35.000,1.477421e+18,1.477422e+18,10262791,45.512215,-73.710059,5.4,2016-10-25T19:05:18-04,1.477437e+18


In [107]:
trip_time_df = processed_trips_df[['id','timestamp_start']].copy()
trip_time_df["DayOfTrip"]= trip_time_df["timestamp_start"].str[:10]

In [108]:
trip_time_df.head()

,id,timestamp_start,DayOfTrip
11,1693055,2016-10-25T18:37:56.000,2016-10-25
14,1629365,2016-10-23T11:24:33.000,2016-10-23
16,1966430,2016-10-27T18:39:24.000,2016-10-27
21,1467160,2016-10-25T12:06:47.000,2016-10-25
23,3488393,2016-10-23T16:26:08.000,2016-10-23


In [109]:
import csv
climate_df = pd.read_csv('Climate.csv', skiprows=25, names=["Date/Time","Year","Month","Day","Data Quality","Max Temp (°C)","Max Temp Flag","Min Temp (°C)","Min Temp Flag","Mean Temp (°C)","Mean Temp Flag","Heat Deg Days (°C)","Heat Deg Days Flag","Cool Deg Days (°C)","Cool Deg Days Flag","Total Rain (mm)","Total Rain Flag","Total Snow (cm)","Total Snow Flag","Total Precip (mm)","Total Precip Flag","Snow on Grnd (cm)","Snow on Grnd Flag","Dir of Max Gust (10s deg)","Dir of Max Gust Flag","Spd of Max Gust (km/h)","Spd of Max Gust Flag"])

In [110]:
climate_df = climate_df.drop(["Max Temp Flag","Day","Month","Year","Min Temp Flag","Data Quality","Mean Temp Flag","Heat Deg Days (°C)","Heat Deg Days Flag","Cool Deg Days (°C)","Cool Deg Days Flag","Total Rain (mm)","Total Rain Flag","Total Snow (cm)","Total Snow Flag","Total Precip (mm)","Total Precip Flag","Snow on Grnd (cm)","Snow on Grnd Flag","Dir of Max Gust (10s deg)","Dir of Max Gust Flag","Spd of Max Gust (km/h)","Spd of Max Gust Flag"], axis = 1) 
climate_df.head()

,Date/Time,Max Temp (°C),Min Temp (°C),Mean Temp (°C)
0,2016-01-01,0.0,-4.0,-2.0
1,2016-01-02,0.4,-4.5,-2.1
2,2016-01-03,0.2,-14.4,-7.1
3,2016-01-04,-12.9,-20.1,-16.5
4,2016-01-05,-6.3,-21.2,-13.8


In [112]:
trip_climate_df = pd.merge(left=trip_time_df,right=climate_df, left_on='DayOfTrip', right_on='Date/Time')
trip_climate_df = trip_climate_df.drop(["timestamp_start", "Date/Time"], axis=1)

In [113]:
trip_climate_df.head()

,id,DayOfTrip,Max Temp (°C),Min Temp (°C),Mean Temp (°C)
0,1693055,2016-10-25,6.9,-0.6,3.2
1,1467160,2016-10-25,6.9,-0.6,3.2
2,1624502,2016-10-25,6.9,-0.6,3.2
3,1761473,2016-10-25,6.9,-0.6,3.2
4,3531134,2016-10-25,6.9,-0.6,3.2


In [116]:
Final_Merged_Data = pd.merge(left=Merged_Trip_Cord_Df,right=trip_climate_df, left_on='trip_id', right_on='id')
Final_Merged_Data = Final_Merged_Data.drop(["id", "DayOfTrip"], axis=1)

In [120]:
Final_Merged_Data.head()

,trip_id,avg_speed,duration,mode,mode_description,trip_timestamp_start,trip_timestamp_end,trip_ftimestamp_start,trip_ftimestamp_end,coordinate_id,latitude,longitude,speed,coordinate_timestamp,coordinate_ftimestamp,Max Temp (°C),Min Temp (°C),Mean Temp (°C)
0,1693055,20.8,1719,2.0,Automobile,2016-10-25T18:37:56.000,2016-10-25T19:06:35.000,1.477421e+18,1.477422e+18,10262787,45.513605,-73.714926,9.0,2016-10-25T19:06:15-04,1.477437e+18,6.9,-0.6,3.2
1,1693055,20.8,1719,2.0,Automobile,2016-10-25T18:37:56.000,2016-10-25T19:06:35.000,1.477421e+18,1.477422e+18,10262788,45.512483,-73.711553,12.9,2016-10-25T19:05:35-04,1.477437e+18,6.9,-0.6,3.2
2,1693055,20.8,1719,2.0,Automobile,2016-10-25T18:37:56.000,2016-10-25T19:06:35.000,1.477421e+18,1.477422e+18,10262789,45.512133,-73.710771,6.0,2016-10-25T19:05:28-04,1.477437e+18,6.9,-0.6,3.2
3,1693055,20.8,1719,2.0,Automobile,2016-10-25T18:37:56.000,2016-10-25T19:06:35.000,1.477421e+18,1.477422e+18,10262790,45.512390,-73.710525,6.5,2016-10-25T19:05:26-04,1.477437e+18,6.9,-0.6,3.2
4,1693055,20.8,1719,2.0,Automobile,2016-10-25T18:37:56.000,2016-10-25T19:06:35.000,1.477421e+18,1.477422e+18,10262791,45.512215,-73.710059,5.4,2016-10-25T19:05:18-04,1.477437e+18,6.9,-0.6,3.2


In [121]:
Final_Merged_Data.tail()

,trip_id,avg_speed,duration,mode,mode_description,trip_timestamp_start,trip_timestamp_end,trip_ftimestamp_start,trip_ftimestamp_end,coordinate_id,latitude,longitude,speed,coordinate_timestamp,coordinate_ftimestamp,Max Temp (°C),Min Temp (°C),Mean Temp (°C)
4987823,3601748,9.8,1593,2.0,Automobile,2016-11-01T17:54:28.000,2016-11-01T18:21:01.000,1.478023e+18,1.478024e+18,18778765,45.538295,-73.731918,14.2,2016-11-01T18:14:25-04,1.478038e+18,11.2,-0.3,5.5
4987824,3601748,9.8,1593,2.0,Automobile,2016-11-01T17:54:28.000,2016-11-01T18:21:01.000,1.478023e+18,1.478024e+18,18778766,45.538745,-73.732351,13.5,2016-11-01T18:14:21-04,1.478038e+18,11.2,-0.3,5.5
4987825,3601748,9.8,1593,2.0,Automobile,2016-11-01T17:54:28.000,2016-11-01T18:21:01.000,1.478023e+18,1.478024e+18,18778767,45.539247,-73.732834,9.9,2016-11-01T18:14:16-04,1.478038e+18,11.2,-0.3,5.5
4987826,3601748,9.8,1593,2.0,Automobile,2016-11-01T17:54:28.000,2016-11-01T18:21:01.000,1.478023e+18,1.478024e+18,18778768,45.539733,-73.732594,-1.0,2016-11-01T18:13:57-04,1.478038e+18,11.2,-0.3,5.5
4987827,3601748,9.8,1593,2.0,Automobile,2016-11-01T17:54:28.000,2016-11-01T18:21:01.000,1.478023e+18,1.478024e+18,18778769,45.540037,-73.732033,11.9,2016-11-01T18:13:55-04,1.478038e+18,11.2,-0.3,5.5
